# ADA Project Milestone P3

## <span style="color: green;">Import libraries and set paths </span>

In [1]:
import os
import sys
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import unidecode as ud
from tqdm import tqdm
from collections import Counter
from datetime import datetime
%matplotlib inline

from preprocessing_functions import *


### Set the global variables for the paths to the data folder

In [2]:
# add your data folder into the 'code' folder, make sure it is in the .gitignore file

CODE_DIR = os.getcwd()
DATA_DIR = os.path.join(CODE_DIR, 'data')

print('your data directory:')
DATA_DIR

your data directory:


'c:\\Users\\fulci\\git\\ada_23\\ada-2023-project-adarescueteam\\code\\data'

In [3]:
# set path for BeerAdvocate data
DATA_BeerAdvocate = os.path.join(DATA_DIR, "BeerAdvocate.tar")

# set path for RateBeer data
DATA_RateBeer = os.path.join(DATA_DIR, "RateBeer.tar")

# set path for MatchedBeer data
DATA_MatchedBeers = os.path.join(DATA_DIR, "matched_beer_data.tar")

## <span style="color: green;">Load cleaned data </span>

## RaterBeer (RB) Data

#### beers, breweries, users csv's

In [4]:
# read in the BeerAdvocate files
BA_beers = pd.read_csv(os.path.join(DATA_BeerAdvocate, "beers.csv"))
BA_breweries = pd.read_csv(os.path.join(DATA_BeerAdvocate, "breweries.csv"))
BA_users = pd.read_csv(os.path.join(DATA_BeerAdvocate, "users.csv"), 
                       converters={"joined": convert_timestamp})

#### ratings pickle

In [5]:
# read RB_ratings from pickle file
with open(os.path.join(DATA_RateBeer, "RB_ratings.pkl"), 'rb') as f:
    RB_ratings = pickle.load(f)


## BeerAdvocate (BA) Data

#### beers, breweries, users csv's

In [6]:
# read in the RateBeer files
RB_beers = pd.read_csv(os.path.join(DATA_RateBeer, "beers.csv"))
RB_breweries = pd.read_csv(os.path.join(DATA_RateBeer, "breweries.csv"))
RB_users = pd.read_csv(os.path.join(DATA_RateBeer, "users.csv"),
                       converters={"joined": convert_timestamp})

#### ratings pickle

In [7]:
# read BA_ratings from pickle file
with open(os.path.join(DATA_BeerAdvocate, "BA_ratings.pkl"), 'rb') as f:
    BA_ratings = pickle.load(f)

## Glutenfree (gf) Beer Data

In [8]:
# read in the glutenfree beers
rb_gf_ratings=pd.read_csv(os.path.join(DATA_RateBeer, 'rb_gf_ratings.csv'), low_memory=False, encoding='utf-8')
ba_gf_ratings=pd.read_csv(os.path.join(DATA_BeerAdvocate, 'ba_gf_ratings.csv'), low_memory=False, encoding='utf-8')
rb_gf_ratings.shape, ba_gf_ratings.shape

((3770, 18), (2397, 19))

In [9]:
# merge the two datasets
frames = [rb_gf_ratings, ba_gf_ratings]
rb_ba_gf_ratings = pd.concat(frames)
rb_ba_gf_ratings.shape

(6167, 19)

In [10]:
rb_ba_gf_ratings.duplicated().sum()

0

## <span style="color: green;">Filter data for matching </span>

### Filter by style

## <span style="color: green;">Merge the RB and BA dataframe after filtering </span>

shrink down the datasets before matching non-glutenfree reviews with glutenfree reviews, takes too long to work else

and not to forget: eliminate duplicates, because most likely as we dont consider the Matched_beer dataset => done


In [11]:
# add review columns to RB_ratings before joining them. all entries have reviews
RB_ratings['review']=True

In [17]:
frames = [BA_ratings, RB_ratings]
RB_BA_ratings = pd.concat(frames)
RB_BA_ratings.shape
# takes 3s

(15515106, 17)

In [13]:
RB_BA_ratings.duplicated().sum()
# takes 35s

18

In [18]:
RB_BA_ratings = RB_BA_ratings[~RB_BA_ratings.duplicated()]
# takes 35s

In [19]:
RB_BA_ratings.duplicated().sum()
# takes again 35s -> this is why we need to shrink down the data first

0

## <span style="color: green;">Temporal analyis</span>

-> can fill with a lot of P2_AfterMilestone from the 'fix_nlp_and_text-formats' branch 

## <span style="color: green;">Language processing</span>

suggest that we write functions / pipeline in an outside script and in here just leave the important things

-> can fill with a lot of P2_AfterMilestone from the 'fix_nlp_and_text-formats' branch 

-> dont forget to do the analysis on the whole rb_ba_gf dataset!

### Language identification

-> can fill with a lot of P2_AfterMilestone from the 'fix_nlp_and_text-formats' branch , need to see what to keep though

### Adjective and adverbs extraction

## <span style="color: lightgreen;">correlation of the reviews with the ratings</span> TBD
